# WA Languages Diversity 

## We will start off by creating different dfs
1. Counties
    - Has all of the county to district information
2. Ancentory and language spoken at home
    - Has the ancestory and language spoken at home for each county
3. School enrollment per county
    - Enrollment by age and educational attainment by county

In [115]:
import pandas as pd

In [101]:
counties = pd.read_csv("data/Counties.csv", header=2)
counties = counties.loc[counties["State Postal Code"] == "WA"]
ppe = pd.read_csv('data/SchoolDistrictPPEAverages.csv')

In [102]:
def clean(series):
    '''
    A function to clean columns in a dataframe to all lowercase strings.
    Returns a cleaned list of the given column.
    '''
    cleaned = []
    for row in series:
        cleaned.append(str(row).lower())
    return cleaned

In [103]:
ppe['district_name'] = clean(ppe['Districtname'])
counties['district_name'] = clean(counties['School District Name'])

In [104]:
merged = ppe.merge(counties, right_on="district_name", left_on="district_name")
merged

,Districtname,PPE,district_name,State Postal Code,State FIPS,District ID Number,School District Name,County Names,County FIPS
0,Aberdeen School District,6191.511824,aberdeen school district,WA,53,30,Aberdeen School District,Grays Harbor County,27
1,Adna School District,14105.516930,adna school district,WA,53,60,Adna School District,Lewis County,41
2,Almira School District,8644.918222,almira school district,WA,53,90,Almira School District,Grant County,25
3,Almira School District,8644.918222,almira school district,WA,53,90,Almira School District,Lincoln County,43
4,Anacortes School District,9780.410885,anacortes school district,WA,53,150,Anacortes School District,Skagit County,57
...,...,...,...,...,...,...,...,...,...
353,Wishram School District,7323.477110,wishram school district,WA,53,10020,Wishram School District,Klickitat County,39
354,Woodland School District,9636.873132,woodland school district,WA,53,10050,Woodland School District,Clark County,11
355,Woodland School District,9636.873132,woodland school district,WA,53,10050,Woodland School District,Cowlitz County,15
356,Yakima School District,9331.332774,yakima school district,WA,53,10110,Yakima School District,Yakima County,77


In [105]:
languages = pd.read_csv("data/S1601LanguagesSpokenAtHome.csv")
languages.set_index('Label (Grouping)', inplace=True)
languages = languages.transpose()
languages['temp_index'] = languages.index
regex_pattern = r"^[\w,\s]+\!\![\w,\s]+\!\![\w,\s]+$"
languages = languages.loc[languages['temp_index'].str.contains(regex_pattern)]
languages.insert(0, "County", "")       
languages.insert(1, "Ratio", "")

for row in languages['temp_index']:
    terms = row.split("!!")
    terms[0] = terms[0].split(",")[0]
    languages.loc[row, ['County']] = terms[0]
    languages.loc[row, ['Ratio']] = terms[1]

languages.rename_axis(None, axis=1, inplace=True)
languages.set_index(['County', 'Ratio'], inplace=True)
languages.drop(columns=['temp_index'], inplace=True, axis=1)


In [106]:
ancestry = pd.read_csv("data/DP02AncestryLanguagesSpokenAtHome.csv")
ancestry.set_index('Label (Grouping)', inplace=True)
ancestry = ancestry.transpose()
ancestry['temp_index'] = ancestry.index
regex_pattern = r"^[\w,\s]+\!\![\w,\s]+$"
ancestry = ancestry.loc[ancestry['temp_index'].str.contains(regex_pattern)]
ancestry.insert(0, "County", "")
ancestry.insert(1, "Ratio", "")

for row in ancestry['temp_index']:
    terms = row.split("!!")
    terms[0] = terms[0].split(",")[0]
    ancestry.loc[row, ['County']] = terms[0]
    ancestry.loc[row, ['Ratio']] = terms[1]

ancestry.rename_axis(None, axis=1, inplace=True)
ancestry.set_index(['County', 'Ratio'], inplace=True)
ancestry.drop(columns=['temp_index'], inplace=True, axis=1)
ancestry.head(30)

ancestry[ancestry.filter(like='With computers').columns]
ancestry


HOUSEHOLDS BY TYPE     Total households  \
County         Ratio                                              
Washington     Estimate                NaN            2,979,272   
               Percent                 NaN            2,979,272   
Adams County   Estimate                NaN                6,304   
               Percent                 NaN                6,304   
Asotin County  Estimate                NaN                9,499   
...                                    ...                  ...   
Whatcom County Percent                 NaN               91,171   
Whitman County Estimate                NaN               17,963   
               Percent                 NaN               17,963   
Yakima County  Estimate                NaN               85,558   
               Percent                 NaN               85,558   

                                Married-couple household  \
County         Ratio                                       
Washington     Estimate                        1,482,230   
               Percent                             49.8%   
Adams County   Estimate                            3,509   
               Percent                             55.7%   
Asotin County  Estimate                            4,180   
...                                                  ...   
Whatcom County Percent                             48.0%   
Whitman County Estimate                            7,021   
               Percent                             39.1%   
Yakima County  Estimate                           42,367   
               Percent                             49.5%   

                                    With children of the householder under 18 years  \
County         Ratio                                                                  
Washington     Estimate                                            588,901            
               Percent                                               19.8%            
Adams County   Estimate                                              1,679            
               Percent                                               26.6%            
Asotin County  Estimate                                              1,264            
...                                                                    ...            
Whatcom County Percent                                               17.8%            
Whitman County Estimate                                              2,708            
               Percent                                               15.1%            
Yakima County  Estimate                                             17,764            
               Percent                                               20.8%            

                                Cohabiting couple household  \
County         Ratio                                          
Washington     Estimate                             251,755   
               Percent                                 8.5%   
Adams County   Estimate                                 500   
               Percent                                 7.9%   
Asotin County  Estimate                                 876   
...                                                     ...   
Whatcom County Percent                                 9.2%   
Whitman County Estimate                               1,532   
               Percent                                 8.5%   
Yakima County  Estimate                               7,859   
               Percent                                 9.2%   

                                    With children of the householder under 18 years  \
County         Ratio                                                                  
Washington     Estimate                                             75,364            
               Percent                                                2.5%            
Adams County   Estimate                                                

In [107]:
enrollment = pd.read_csv("data/S1401SchoolEnrollment.csv")
enrollment.set_index('Label (Grouping)', inplace=True)
enrollment = enrollment.transpose()
enrollment['temp_index'] = enrollment.index
regex_pattern = r"^[\w,\s]+\!\![\w,\s]+\!\![\w,\s]+$"
enrollment = enrollment.loc[enrollment['temp_index'].str.contains(regex_pattern)]
enrollment.insert(0, "County", "")
enrollment.insert(1, "Ratio", "")

for row in enrollment['temp_index']:
    terms = row.split("!!")
    terms[0] = terms[0].split(",")[0]
    enrollment.loc[row, ['County']] = terms[0]
    enrollment.loc[row, ['Ratio']] = terms[1]

enrollment.rename_axis(None, axis=1, inplace=True)
enrollment.set_index(['County', 'Ratio'], inplace=True)
enrollment.drop(columns=['temp_index'], inplace=True, axis=1)
enrollment.head(30)

Population 3 years and over enrolled in school  \
County        Ratio                                                                      
Washington    Total                                                          1,766,556   
              Percent                                                              (X)   
              In public school                                                     (X)   
              Percent in public school                                           84.0%   
              In private school                                                    (X)   
              Percent in private school                                          16.0%   
Adams County  Total                                                              6,133   
              Percent                                                              (X)   
              In public school                                                     (X)   
              Percent in public school                                           95.3%   
              In private school                                                    (X)   
              Percent in private school                                           4.7%   
Asotin County Total                                                              4,221   
              Percent                                                              (X)   
              In public school                                                     (X)   
              Percent in public school                                           88.2%   
              In private school                                                    (X)   
              Percent in private school                                          11.8%   
Benton County Total                                                             53,279   
              Percent                                                              (X)   
              In public school                                                     (X)   
              Percent in public school                                           87.5%   
              In private school                                                    (X)   
              Percent in private school                                          12.5%   
Chelan County Total                                                             17,955   
              Percent                                                              (X)   
              In public school                                                     (X)   
              Percent in public school                                           87.9%   
              In private school                                                    (X)   
              Percent in private school                                          12.1%   

                                            Nursery school, preschool  \
County        Ratio                                                     
Washington    Total                                           107,091   
              Percent                                            6.1%   
              In public school                                 51,222   
              Percent in public school                          47.8%   
              In private school                                55,869   
              Percent in private school                         52.2%   
Adams County  Total                                               238   
              Percent                                            3.9%   
              In public school                                    218   
              Percent in public school                          91.6%   
              In private school                                    20   
              Percent in private school                          8.4%   
Asotin County Total                                               387   
              Percent                                           

In [108]:
education = pd.read_csv('data/S1501EducationByRace.csv')
education.set_index('Label (Grouping)', inplace=True)
education = education.transpose()
education['temp_index'] = education.index
regex_pattern = r"^[\w,\s]+\!\![\w,\s]+\!\![\w,\s]+$"
education = education.loc[education['temp_index'].str.contains(regex_pattern)]
education.insert(0, "County", "")
education.insert(1, "Ratio", "")

for row in education['temp_index']:
    terms = row.split("!!")
    terms[0] = terms[0].split(",")[0]
    education.loc[row, ['County']] = terms[0]
    education.loc[row, ['Ratio']] = terms[1]

education.rename_axis(None, axis=1, inplace=True)
education.set_index(['County', 'Ratio'], inplace=True)
education.drop(columns=['temp_index'], inplace=True, axis=1)
education.head(30)

education

AGE BY EDUCATIONAL ATTAINMENT  \
County        Ratio                                          
Washington    Total                                    NaN   
              Percent                                  NaN   
              Male                                     NaN   
              Percent Male                             NaN   
              Female                                   NaN   
...                                                    ...   
Yakima County Percent                                  NaN   
              Male                                     NaN   
              Percent Male                             NaN   
              Female                                   NaN   
              Percent Female                           NaN   

                                 Population 18 to 24 years  \
County        Ratio                                          
Washington    Total                                671,550   
              Percent                                  (X)   
              Male                                 351,705   
              Percent Male                             (X)   
              Female                               319,845   
...                                                    ...   
Yakima County Percent                                  (X)   
              Male                                  12,769   
              Percent Male                             (X)   
              Female                                12,171   
              Percent Female                           (X)   

                                     Less than high school graduate  \
County        Ratio                                                   
Washington    Total                                          85,472   
              Percent                                         12.7%   
              Male                                           50,429   
              Percent Male                                    14.3%   
              Female                                         35,043   
...                                                             ...   
Yakima County Percent                                         25.4%   
              Male                                            3,853   
              Percent Male                                    30.2%   
              Female                                          2,482   
              Percent Female                                  20.4%   

                                     High school graduate (includes equivalency)  \
County        Ratio                                                                
Washington    Total                                                     244,136    
              Percent                                                     36.4%    
              Male                                                      138,950    
              Percent Male                                                39.5%    
              Female                                                    105,186    
...                                                                         ...    
Yakima County Percent                                                     43.1%    
              Male                                                        6,173    
              Percent Male                                                48.3%    
              Female                                                      4,573    
              Percent Female                                              37.6%    

                                     Some college or associate's degree  \
County        Ratio                                                       
Washington    Total                                             235,801   
              Percent                                             35.1%   
              Male                                              117,715   
              Percent M

# Research Questions:
1. How does language diversity in each Washington county affect their educational attainment outcomes? S1501, S1401 (2)

2. How do school district budgets and spending affect spoken language diversity in Washington counties? DOWNLOADED ONLINE (2)

3. To what extent do Washington residents continue to practice their ancestral culture through language? DP02 (1)


In [109]:
# 1. how does language diversity in each Washington county affect their educational attainment outcomes?
# make a df that has the different counties as the index and the language diversity as columns
languages = languages.loc[(languages.index.get_level_values("County"), "Percent"), :]
columns = []
columns.append(languages.filter(like="Speak a language other than English").columns[0])
columns.append(languages.filter(like="Spanish").columns[0])
columns.append(languages.filter(like="Other Indo-European languages").columns[0])
columns.append(languages.filter(like="Asian and Pacific Island languages").columns[0])
columns.append(languages.filter(like="Other languages").columns[0])


enrollment = enrollment.loc[(enrollment.index.get_level_values("County"), "Percent"), :]
columns2 = []
columns2.append(enrollment.filter(like="College, undergraduate").columns[0])
columns2.append(enrollment.filter(like="Graduate, professional school").columns[0])

enrollment = enrollment.droplevel(1)[columns2]
languages = languages.droplevel(1)[columns]

q1 = pd.merge(enrollment, languages, on=['County'])
q1

,"College, undergraduate","Graduate, professional school",Speak a language other than English,Spanish,Other Indo-European languages,Asian and Pacific Island languages,Other languages
County,,,,,,,
Washington,20.6%,5.3%,20.5%,8.7%,4.2%,6.3%,1.4%
Adams County,8.7%,0.8%,51.1%,48.9%,0.8%,0.5%,0.9%
Asotin County,13.7%,1.0%,4.4%,2.5%,0.5%,1.3%,0.1%
Benton County,15.1%,3.6%,21.4%,16.3%,2.3%,1.9%,0.9%
Chelan County,17.3%,3.3%,26.1%,24.1%,1.3%,0.7%,0.0%
Clallam County,16.7%,3.8%,5.7%,3.1%,1.4%,0.8%,0.4%
Clark County,16.7%,3.6%,15.7%,6.3%,5.5%,3.4%,0.6%
Columbia County,23.1%,5.1%,6.6%,5.6%,0.2%,0.8%,0.0%
Cowlitz County,18.1%,2.8%,7.7%,5.3%,1.2%,1.0%,0.1%


In [110]:
# 2. how do school district budgets and spending affect spoken langugae diversity in WA counties?
# make a df that has the different counties as the index and the 
# language diversity as columns (in number like total of 40 different languages)
# make another column that has the budget of all the school districts.
#  budget can either be determined by the sum of all budgets in a county or the average of the budgets.
merged[["PPE", "County Names"]].groupby("County Names").mean()

,PPE
County Names,
Adams County,6819.601980
Asotin County,8730.831671
Benton County,10088.104721
Chelan County,6645.849536
Clallam County,9487.435213
Clark County,10641.747868
Columbia County,11009.470113
Cowlitz County,13053.507035
Douglas County,7499.022128


In [111]:
# 3. to what extent is a household's lingual abilities a factor of at-home computer accessibility?
# we make a df that has the different language categetories as the index, 
# and if there is a computer use and if there is broadband access fr the columns and them the percentage. 

In [139]:
mini_ancestry = ancestry.iloc[:, -32:].copy().loc[(mini_ancestry.index.get_level_values("County"), "Estimate"), :]
mini_ancestry.drop(columns="COMPUTERS AND INTERNET USE", inplace=True)
mini_ancestry = mini_ancestry.apply(lambda x: x.str.replace(',',''))

In [137]:
import unicodedata
import statsmodels.formula.api as smf


In [114]:
mini_ancestry.columns = [name.strip() for name in mini_ancestry.columns]
mini_ancestry['internet_ratio'] = mini_ancestry["With a broadband Internet subscription"].astype(float) / mini_ancestry["Total households"].astype(float)
mini_ancestry['computer_ratio'] = mini_ancestry["With a computer"].astype(float) / mini_ancestry["Total households"].astype(float)
mini_ancestry

,,Total population,American,Arab,Czech,Danish,Dutch,English,French (except Basque),French Canadian,German,...,Swedish,Swiss,Ukrainian,Welsh,West Indian (excluding Hispanic origin groups),Total households,With a computer,With a broadband Internet subscription,internet_ratio,computer_ratio
County,Ratio,,,,,,,,,,,,,,,,,,,,,
Washington,Estimate,7688549,285061,39659,27570,61497,123131,868037,197049,44181,1152811,...,203180,30910,64073,65396,16398,2979272,2864529,2743870,0.920987,0.961486
Adams County,Estimate,20557,409,42,22,40,141,1109,195,43,1844,...,190,16,17,36,0,6304,5663,5360,0.850254,0.898319
Asotin County,Estimate,22370,1291,0,11,153,301,3028,552,187,4517,...,513,60,0,121,0,9499,8964,8336,0.877566,0.943678
Benton County,Estimate,207560,9252,1385,737,2112,4675,29694,4937,956,35195,...,4771,605,724,2593,127,75509,72194,68181,0.902952,0.956098
Chelan County,Estimate,79076,4521,121,350,473,1343,8555,1843,355,13510,...,1889,263,91,597,0,30414,28564,26757,0.879759,0.939173
Clallam County,Estimate,77333,3564,190,215,516,1623,12220,2477,557,13774,...,2455,521,297,1083,130,34128,32044,30844,0.903774,0.938936
Clark County,Estimate,504091,21510,1976,1903,3958,8833,62773,13909,4012,83546,...,14510,2537,8985,4531,196,188863,182440,176036,0.932083,0.965991
Columbia County,Estimate,3980,258,0,0,59,123,602,107,18,756,...,52,35,4,9,0,1804,1698,1366,0.757206,0.941242
Cowlitz County,Estimate,110621,5205,124,197,914,2531,14287,3332,842,20085,...,2825,430,92,517,287,43167,40551,38128,0.883267,0.939398
